<a href="https://colab.research.google.com/github/IvanovMaxim2000/Homework_AI/blob/main/ML_Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №3
## Numba
### Постановка задачи:


1.   С помощью функционала библиотеки Numba ускорить работу градиентного спуска и его модификаций
2.   Продемонстрировать прирост производительности



### Импортирование библиотек

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from numpy import linalg as la
import numba
from datetime import datetime

In [ ]:
class But:

  @staticmethod
  def func(x : np.array):
    return (x[0] + 2*x[1] -7)**2 + (2*x[0] + x[1] - 5)**2

  @staticmethod
  def dx(x : np.array):
    return 6*x[0];

  @staticmethod
  def dy(x : np.array):
    return 6*x[1];

  @staticmethod
  def glob_min():
      return np.array([1,3,0])
    
  @staticmethod
  def start_point():
      return np.array([-10, 5])

In [ ]:
class fast_But:

    @staticmethod
    @numba.njit(fastmath = True)
    def func(x : np.array):
      return (x[0] + 2*x[1] -7)**2 + (2*x[0] + x[1] - 5)**2

    @staticmethod
    @numba.njit(fastmath = True)
    def dx(x : np.array):
      return 6*x[0];

    @staticmethod
    @numba.njit(fastmath = True)
    def dy(x : np.array):
      return 6*x[1];

    @staticmethod
    @numba.njit(fastmath = True)
    def glob_min():
      return np.array([1,3,0])

    @staticmethod
    @numba.njit(fastmath = True)
    def start_point():
      return np.array([-10, 5])

In [ ]:
class Matiac:

  @staticmethod
  def func(x : np.array):
    return 0.26*(x[0]*x[0] +x[1]*x[1])-0.48*x[0]*x[1]

  @staticmethod
  def dx(x : np.array):
    return 0.52*x[0] - 0.48 * x[1]

  @staticmethod
  def dy(x : np.array):
    return 0.52*x[1] - 0.48 * x[0]

  @staticmethod
  def glob_min():
      return np.array([0,0,0])

  @staticmethod
  def start_point():
      return np.array([-10, 5])


In [ ]:
class fast_Matiac:

  @staticmethod
  @numba.njit(fastmath = True)
  def func(x : np.array):
    return 0.26*(x[0]*x[0] +x[1]*x[1])-0.48*x[0]*x[1]

  @staticmethod
  @numba.njit(fastmath = True)
  def dx(x : np.array):
    return 0.52*x[0] - 0.48 * x[1]

  @staticmethod
  @numba.njit(fastmath = True)
  def dy(x : np.array):
    return 0.52*x[1] - 0.48 * x[0]

  @staticmethod
  @numba.njit(fastmath = True)
  def glob_min():
      return np.array([0,0,0])

  @staticmethod
  @numba.njit(fastmath = True)
  def start_point():
      return np.array([-10, 5])


### Функция для сравнения обычного и ускоренного методов

In [ ]:
import time

In [ ]:
def time_compare(func, fast_func, GD, fast_GD, x0,y0):
    start = time.time()
    grad_history_GD = GD(func, x0, y0, 0.1, 250)
    end_1 = time.time() - start
    
    start = time.time()
    grad_history_fast_GD = fast_GD(fast_func.func, fast_func.dx, fast_func.dy, fast_func.start_point())
    end_2 = time.time() - start
    print(f"Time without Numba:{end_1}\nTime with Numba:{end_2}")
    print(f"GD solution: {grad_history_GD[-1]}\nFast_modification solution: {grad_history_fast_GD[-1]} ")


## Классический градиентный спуск

In [ ]:
def GD( f, x0, y0, lr, max_iterations, delta = 0.001):

    '''
    f - функция оптимизации
    x0,y0 - начальная точка
    lr - learning rate(0.1)
    max_iterations - максимальное количество итераций)
    '''
    grad_history = [np.array([x0, y0, f.func(np.array([x0,y0]))])]
    i = 0
    while (i < max_iterations and la.norm(grad_history[-1] - f.glob_min()) > delta):
        i += 1
        new_x = x0 - lr * f.dx(np.array([x0,y0]))
        new_y = y0 - lr * f.dy(np.array([x0,y0]))
        x0 = new_x
        y0 = new_y
        grad_history.append(np.array([x0, y0, f.func(np.array([x0,y0]))]))
       
    return grad_history

### Ускоренный классический градиентный спуск

In [ ]:
@numba.njit(fastmath = True)
def faster_GD(func, dx, dy, start_point: np.array, glob_min, lr = 0.1, max_iterations = 50, delta = 0.001) -> np.array:

    '''
    f - функция оптимизации
    x0,y0 - начальная точка
    lr - learning rate(0.1)
    max_iterations - максимальное количество итераций)
    '''

    current_point = start_point.copy()
    grad_history = [np.array([current_point[0], current_point[1], func(current_point)])]
    i = 0
    while (i < max_iterations):
        current_point[0] = start_point[0] - lr * dx(start_point)
        current_point[1] = start_point[1] - lr * dy(start_point)
        start_point[0] = current_point[0]
        start_point[1] = current_point[1]
        grad_history.append(np.array([start_point[0], start_point[1], func(start_point)]))
        i += 1
       
    return grad_history

### Результаты времени работы с применением Numba и без

In [ ]:
x, y = -10, 5
%timeit -n100 grad_history = GD(But, x, y, 0.1, 250)

4.11 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit -n100 grad_history = faster_GD(fast_But.func, fast_But.dx, fast_But.dy, fast_But.start_point(), fast_But.glob_min())

The slowest run took 14.16 times longer than the fastest. This could mean that an intermediate result is being cached.
633 µs ± 983 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
x , y = -10, 5
time_compare(But, fast_But, GD, faster_GD, x, y)

Time without Numba:0.007699012756347656
Time with Numba:0.00500798225402832
GD solution: [-3.27339061e-99  1.63669530e-99  7.40000000e+01]
Fast_modification solution: [ 0  0 74] 


## Моментный градиентный спуск

In [ ]:
def MGD( f, x0, y0, lr, max_iterations, delta = 0.001):

    '''
    f - функция оптимизации
    x0,y0 - начальная точка
    lr - learning rate(0.1)
    max_iterations - максимальное количество итераций)
    '''
    grad_history = [np.array([x0, y0, f.func(np.array([x0,y0]))])]
    gamma = 0.7
    i = 0
    x_prev = x0
    y_prev = y0
    while (i < max_iterations):
        i += 1
        new_x = x0 - lr * f.dx(np.array([x0,y0])) + gamma * (x0 - x_prev)
        new_y = y0 - lr * f.dy(np.array([x0,y0])) + gamma * (y0 - y_prev)
        x_prev = x0
        y_prev = y0 
        x0 = new_x
        y0 = new_y
        grad_history.append(np.array([x0, y0, f.func(np.array([x0,y0]))]))
       
    return grad_history

## Ускоренный моментный градиентный спуск

In [ ]:
@numba.njit(fastmath = True)
def fast_MGD( func, dx, dy, start_point, lr = 0.1, max_iterations = 250, delta = 0.001):

    '''
    f - функция оптимизации
    x0,y0 - начальная точка
    lr - learning rate(0.1)
    max_iterations - максимальное количество итераций)
    '''

    grad_history = [np.array([start_point[0], start_point[1], func(start_point)])]
    gamma = 0.7
    i = 0
    current_point = start_point.copy()
    prev_point = start_point.copy()
    while (i < max_iterations):
        current_point[0] = start_point[0] - lr * dx(start_point) + gamma * (start_point[0] - prev_point[0])
        current_point[1] = start_point[1] - lr * dy(start_point) + gamma * (start_point[1] - prev_point[1])
        prev_point[0] = start_point[0]
        prev_point[1] = start_point[1] 
        start_point[0] = current_point[0]
        start_point[1] = current_point[1]
        grad_history.append(np.array([start_point[0], start_point[1], func(start_point)]))
        i += 1
       
    return grad_history

## Сравнение времени работы с применением Numba и без

In [ ]:
x, y = -10, 5
%timeit -n100 grad_history = MGD(But, x, y, 0.1, 250)

1.91 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
x, y = -10, 5
%timeit -n100 fast_MGD(fast_But.func, fast_But.dx, fast_But.dy, fast_But.start_point())

403 µs ± 45.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
x , y = -10, 5
time_compare(But, fast_But, MGD, fast_MGD, x, y)

Time without Numba:0.003389596939086914
Time with Numba:0.0010728836059570312
GD solution: [-4.45693248e-19  2.22846624e-19  7.40000000e+01]
Fast_modification solution: [ 0  0 74] 


## Адаптивный градиентный спуск

In [ ]:
def AGD( f, x0, y0, lr = 0.1, max_iterations = 250, b1 = 0.6, b2 = 0.999, e = 10e-8, delta = 0.001):

    '''
    f - функция оптимизации
    x0,y0 - начальная точка
    lr - learning rate(0.1)
    max_iterations - максимальное количество итераций)
    b1 - коэф. beta1
    b2 - коэф. beta2
    delta - радиус сходимости
    '''
    grad_history = [np.array([x0, y0, f.func(np.array([x0,y0]))])]
    momentum_1 = np.array([0,0])
    momentum_2 = np.array([0,0])
    i = 0
    while (i < max_iterations):
        i += 1
        momentum_1 = b1 * momentum_1 + (1 - b1) * np.array([f.dx(np.array([x0,y0])), f.dy(np.array([x0,y0]))]) 
        momentum_2 = b2 * momentum_2 + (1 - b2) * np.array([f.dx(np.array([x0,y0])), f.dy(np.array([x0,y0]))]) **2
        x0 = x0 - lr * momentum_1[0] / (np.sqrt(momentum_2[0]) + e)
        y0 = y0 - lr * momentum_1[1] / (np.sqrt(momentum_2[1]) + e)
        grad_history.append(np.array([x0, y0, f.func(np.array([x0,y0]))]))
       
    return grad_history

## Ускоренный адаптивный градиентный спуск

In [ ]:
@numba.njit(fastmath = True)
def fast_AGD( func, dx, dy, start_point, lr = 0.1, max_iterations = 250, b1 = 0.6, b2 = 0.999, e = 10e-8, delta = 0.001):

    '''
    f - функция оптимизации
    x0,y0 - начальная точка
    lr - learning rate(0.1)
    max_iterations - максимальное количество итераций)
    b1 - коэф. beta1
    b2 - коэф. beta2
    delta - радиус сходимости
    '''
    grad_history = [np.array([start_point[0], start_point[1], func(start_point)])]
    current_point = start_point.copy()
    momentum_1 = np.array([0.,0.])
    momentum_2 = np.array([0.,0.])
    i = 0
    while (i < max_iterations):
        momentum_1 = b1 * momentum_1 + (1 - b1) * np.array([dx(current_point), dy(current_point)]) 
        momentum_2 = b2 * momentum_2 + (1 - b2) * np.array([dx(current_point), dy(current_point)]) **2
        current_point[0] = current_point[0] - lr * momentum_1[0] / (np.sqrt(momentum_2[0]) + e)
        current_point[1] = current_point[1] - lr * momentum_1[1] / (np.sqrt(momentum_2[1]) + e)
        grad_history.append(np.array([current_point[0], current_point[1], func(current_point)]))
        i += 1
       
    return grad_history

## Сравнение времени работы с Numba и без

In [ ]:
x, y = -10, 5
%timeit -n100 grad_history = MGD(But, x, y, 0.1, 250)

1.85 ms ± 64.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit -n100 fast_AGD(fast_But.func, fast_But.dx, fast_But.dy, fast_But.start_point())

525 µs ± 55.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
x , y = -10, 5
time_compare(But, fast_But, AGD, fast_AGD, x, y)

Time without Numba:0.011733055114746094
Time with Numba:0.0018146038055419922
GD solution: [-2.34068379e-27 -6.91088011e-29  7.40000000e+01]
Fast_GD solution: [ 0  0 74] 
